In [49]:
from graphmaker import graph, cabinet

import numpy as np
import pandas as pd

# from pyvis.network import Network
import networkx as nx
# from fa2 import ForceAtlas2
import igraph as ig
import matplotlib.pyplot as plt
from pydantic import BaseModel

from dataclasses import dataclass

In [54]:
class Node(BaseModel):
    key: str
    label: str
    tag: str
    URL: str
    cluster: str
    x: float
    y: float
    score: float
    
    class Config:
        frozen = True
    
class Cluster(BaseModel):
    key: str
    color: str
    clusterLabel: str
    
    class Config:
        frozen = True
    
    
class Tag(BaseModel):
    key: str
    image: str
    

class Response(BaseModel):
    nodes: list[Node]
    edges: list[list]
    clusters: list[Cluster]
    tags: list[Tag]


In [17]:

g = graph.make_graph()
g = nx.relabel.convert_node_labels_to_integers(g, label_attribute="node_id")

h = ig.Graph.from_networkx(g)
layout = h.layout_fruchterman_reingold(weights="width", grid=False, niter=2000)

In [75]:
tags = [Tag(key="Person", image="person.svg"),
        Tag(key="Tool", image="tool.svg")]

edges = []

exists_nodes = set()
nodes = set()
clusters = set()
for source, target, data in g.edges(data=True):
    source_node = g.nodes[source]
    target_node = g.nodes[target]
    
    edges.append([source, target])
    
    x_source, y_source = layout[source]
    x_target, y_target = layout[target]
    
    source_score = min(max([g.degree(source) / 2, 2]), 10)
    target_score = min(max([g.degree(target) / 2, 2]), 10) 
    
    if source not in exists_nodes:
        source_node = Node(key=source, 
                        label=source_node['label'],
                        cluster=data['group_id'],
                        x=x_source,
                        y=y_source,
                        tag='Person',
                        score=source_score,
                        URL="google.com")
        nodes.add(source_node)
        exists_nodes.add(source)
        
    if target not in exists_nodes:
        target_node = Node(key=target, 
                        label=target_node['label'],
                        cluster=data['group_id'],
                        x=x_target,
                        y=y_target,
                        tag='Tool',
                        score=target_score,
                        URL="google.com")
        nodes.add(target_node)
        exists_nodes.add(target)
    
    cluster = Cluster(key=data["group_id"], color=data['color'], clusterLabel=data['group'])
    clusters.add(cluster)

In [76]:
response = Response(nodes=nodes, edges=edges, clusters=clusters, tags=tags)

In [77]:
import json

with open("/mnt/d/Projects/miem-users-network/GraphExplorer/public/miem_dataset.json", "w", encoding="UTF-8") as f:
    json.dump(response.dict(), f, ensure_ascii=False)
    

In [9]:
layout = h.layout_fruchterman_reingold(weights='width', grid=False, niter = 2000)

In [10]:
for node, data in g.nodes(data=True):
    x, y = layout[node]
    data["viz"] = {'size': data['size']}
    data["viz"]["position"] = {"x": x*40, "y": y*40, "z": 0.0} 

In [11]:
[i for i in g.edges(data=True)]

[(0,
  1,
  {'width': 4.83,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  1,
  {'width': 6.41,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  79,
  {'width': 6.13,
   'group': 'Наука',
   'group_id': 1,
   'color': '#005f73',
   'label': 'Наука'}),
 (2,
  458,
  {'width': 4.88,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  462,
  {'width': 4.88,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  464,
  {'width': 4.88,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  468,
  {'width': 4.83,
   'group': 'Неизвестно',
   'group_id': 0,
   'color': '#001219',
   'label': 'Неизвестно'}),
 (2,
  649,
  {'width': 2.29,
   'group': 'Наука',
   'group_id': 1,
   'color': '#005f73',
   'label': 'Наука'}),
 (2,
  779,
  {'width': 1

In [12]:
nx.readwrite.gexf.write_gexf(g, "/mnt/d/Projects/miem-users-network/Frontend/public/graph.gexf")

In [26]:
network = Network(width='100%', height='100%')
# network.toggle_stabilization(False)
network.toggle_physics(False)

for node, data in graph.nodes(data=True):
    x, y = layout[node]
    network.add_node(node, size=max(graph.degree[node] / 2, 4), title=data['title'], color=data['color'], x=x*40, y=y*40)


for source, target, data in graph.edges(data=True):
    network.add_edge(source, target, width=data['width'] / 100)

# network.show_buttons()
network.show("graph.html")

In [13]:
for node, data in graph.nodes(data=True):
    if node == 0:
        print(data, node)

{'name': 'Трофимов Станислав Иванович', 'role': 'Инициатор проекта', 'color': '#6bd61a'} 0
